# **fGraph 0 detection from eigenvalues**


<div id="import_libraries">
    <h2>Import Libraries</h2>
</div>


In [1]:
#!pip install pandas
#!pip install scikit-learn
#!pip install matplotlib
#!pip install xgboost

In [14]:
import gzip

In [15]:
with gzip.open('fgraph_eigen.csv.gz', 'wt') as f:
    df.to_csv(f, index=False)

In [16]:
df.to_csv('fgraph_eigen.csv.xz', index=False, compression='xz')

To import the libraries that will be used in this lab, execute the cells below. 


In [1]:
# Import the libraries we need to use in this lab
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import ast
import time
# model trainin set
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.utils.class_weight import compute_sample_weight
#models 
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBRegressor
#model evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error, r2_score

## Load the dataset




In [ ]:
# load the dataset
path= "fgraph_eigen_10.csv.gz"
# read the input data
#df = pd.read_csv(path)
df = pd.read_csv('your_data.csv.gz', compression='gzip')
df


,COEFFICIENT_DEN,DEN_EIGEN1,DEN_EIGEN2,DEN_EIGEN3,DEN_EIGEN4,DEN_EIGEN5,DEN_EIGEN6,DEN_EIGEN7,DEN_EIGEN8,DEN_EIGEN9,...,FGRAPH_EIGEN5,FGRAPH_EIGEN6,FGRAPH_EIGEN7,FGRAPH_EIGEN8,FGRAPH_EIGEN9,FGRAPH_EIGEN10,FGRAPH_EIGEN11,FGRAPH_EIGEN12,FGRAPH_EIGEN13,FGRAPH_EIGEN14
0,0,-2.361469,-2.308931,-2.000000,-2.000000,-1.812953,-1.069841,-1.0,-0.414214,-0.167449,...,-1.672599,-1.489586,-6.205496e-01,0.140528,0.450277,0.784405,2.575268,3.624631,3.879614,4.000000
1,0,-2.361469,-2.308931,-2.000000,-2.000000,-1.812953,-1.069841,-1.0,-0.414214,-0.167449,...,-1.784126,-1.469224,-8.112717e-01,0.199064,0.396535,0.747539,2.715011,3.529143,3.948302,4.000000
2,0,-2.361469,-2.308931,-2.000000,-2.000000,-1.812953,-1.069841,-1.0,-0.414214,-0.167449,...,-1.696434,-1.298952,-4.487057e-01,0.381614,0.708129,0.870774,2.185187,3.533568,3.893163,4.000000
3,0,-2.361469,-2.308931,-2.000000,-2.000000,-1.812953,-1.069841,-1.0,-0.414214,-0.167449,...,-1.643355,-1.400903,-7.144768e-01,-0.301851,0.420685,0.863772,2.764523,3.312131,3.878584,4.000000
4,0,-2.361469,-2.308931,-2.000000,-2.000000,-1.812953,-1.069841,-1.0,-0.414214,-0.167449,...,-1.938435,-0.935602,-3.758097e-01,-0.202765,0.477886,0.783909,2.642675,3.225566,3.880782,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900140,0,-2.882480,-2.361163,-2.104247,-1.647006,-1.380401,-1.289624,-1.0,0.197168,0.355259,...,-1.422987,-1.408866,-7.946356e-01,0.179409,0.201304,0.745293,2.094890,2.812695,4.000000,4.206691
900141,0,-2.882480,-2.361163,-2.104247,-1.647006,-1.380401,-1.289624,-1.0,0.197168,0.355259,...,-1.408866,-1.241961,-6.958461e-01,0.201304,0.625077,0.745293,1.948906,2.812695,3.989238,4.000000
900142,0,-2.882480,-2.361163,-2.104247,-1.647006,-1.380401,-1.289624,-1.0,0.197168,0.355259,...,-1.480196,-1.021678,-5.436425e-01,0.199224,0.332889,1.094859,2.037597,2.829080,3.833295,4.000000
900143,0,-2.882480,-2.361163,-2.104247,-1.647006,-1.380401,-1.289624,-1.0,0.197168,0.355259,...,-1.408866,-0.953212,1.699417e-16,0.201304,0.459386,0.745293,2.069088,2.812695,3.681677,4.000000


In [3]:
# Let's round up the columns to 6 decimals.
# Take all non-integer coulumns
#cols_to_round = df.columns.difference(['COEFFICIENT_DEN'])

# Round those columns to, say, 6 decimals
#df[cols_to_round] = df[cols_to_round].round(8)
#df

# ChatGpt clasifier computation

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score


# 1. Extract features and target
y = df["COEFFICIENT_DEN"]
X = df.drop(columns=["COEFFICIENT_DEN"])


In [5]:
# 2. Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [6]:
# 3. Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Train a classifier (Logistic Regression here)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_scaled, y_train)

# 5. Predict and evaluate
y_pred = clf.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8665437235112121
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.96      0.92    152742
           1       0.60      0.35      0.44     27287

    accuracy                           0.87    180029
   macro avg       0.75      0.66      0.68    180029
weighted avg       0.85      0.87      0.85    180029



### most important feature

In [7]:
# 7. Get feature importance from Logistic Regression
feature_names = X.columns
coefficients = clf.coef_[0]  # For binary classification, it's a 1D array

# Create a DataFrame for sorting and readability
feature_importance = pd.DataFrame({
    'feature': feature_names,
    'coefficient': coefficients,
    'abs_coefficient': np.abs(coefficients)
})

# Sort by absolute value of coefficients
feature_importance_sorted = feature_importance.sort_values(by='abs_coefficient', ascending=False)

# Display the most relevant feature
most_relevant = feature_importance_sorted.iloc[0:5]
print("Most relevant features:")
print(most_relevant)


Most relevant features:
           feature  coefficient  abs_coefficient
12     DEN_EIGEN13    -1.965938         1.965938
13     DEN_EIGEN14     1.160059         1.160059
26  FGRAPH_EIGEN13    -0.995103         0.995103
0       DEN_EIGEN1     0.901467         0.901467
25  FGRAPH_EIGEN12    -0.806606         0.806606


### Decision tree

In [8]:
# 1. (Optional) scale features – not needed for decision trees but harmless
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 2. Train a Decision Tree classifier
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train_scaled, y_train)

# 3. Predict and evaluate
y_pred = tree_clf.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8525404240427931
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.91      0.91    152742
           1       0.51      0.53      0.52     27287

    accuracy                           0.85    180029
   macro avg       0.71      0.72      0.72    180029
weighted avg       0.85      0.85      0.85    180029



In [11]:
# 4. Feature importances from the decision tree
importances = tree_clf.feature_importances_
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print("Most important features:")
print(feature_importance.iloc[0:4])

Most important features:
           feature  importance
12     DEN_EIGEN13    0.130544
26  FGRAPH_EIGEN13    0.123900
25  FGRAPH_EIGEN12    0.075816
13     DEN_EIGEN14    0.053052


### Random forest

In [10]:
from sklearn.ensemble import RandomForestClassifier

# 1. Train the Random Forest
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train_scaled, y_train)

# 2. Predict and evaluate
y_pred_rf = rf_clf.predict(X_test_scaled)

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Classification Report:\n", classification_report(y_test, y_pred_rf))

Random Forest Accuracy: 0.8949724766565387
Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.97      0.94    152742
           1       0.76      0.45      0.56     27287

    accuracy                           0.89    180029
   macro avg       0.83      0.71      0.75    180029
weighted avg       0.89      0.89      0.88    180029



In [12]:
# 3. Feature importances from the random forest
importances = rf_clf.feature_importances_
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print("Most important features:")
print(feature_importance.iloc[0:4])

Most important features:
           feature  importance
26  FGRAPH_EIGEN13    0.107243
12     DEN_EIGEN13    0.095868
25  FGRAPH_EIGEN12    0.070756
22   FGRAPH_EIGEN9    0.045262


# Can the model fit the data?

Let's use the 100% of our data to train the model. Does a solution exist? Let's try the logistic regression

In [13]:
# Prepare features and labels
X = df.drop(columns=["COEFFICIENT_DEN"])
y = df["COEFFICIENT_DEN"]

# Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train on full data
clf = LogisticRegression(max_iter=1000)
clf.fit(X_scaled, y)

# Evaluate on same data
y_pred = clf.predict(X_scaled)
print("Accuracy on full dataset:", accuracy_score(y, y_pred))


Accuracy on full dataset: 0.8671714001633071


In [13]:
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_scaled, y)

y_pred_rf = rf_clf.predict(X_scaled)
print("Random Forest accuracy:", accuracy_score(y, y_pred_rf))

Random Forest accuracy: 0.9999900016108516


In [14]:
importances = rf_clf.feature_importances_
feat_importance = pd.DataFrame({'feature': X.columns, 'importance': importances})
feat_importance = feat_importance.sort_values(by='importance', ascending=False)

print("Top features:\n", feat_importance.head())


Top features:
            feature  importance
26  FGRAPH_EIGEN13    0.106053
12     DEN_EIGEN13    0.097115
25  FGRAPH_EIGEN12    0.072657
22   FGRAPH_EIGEN9    0.045402
10     DEN_EIGEN11    0.043847


In [15]:
# 5. Find misclassified samples
misclassified = df[y != y_pred_rf]

print(f"Number of misclassified samples: {len(misclassified)}")
print("Misclassified indices:")
print(misclassified.index.tolist())

Number of misclassified samples: 9
Misclassified indices:
[4724, 7075, 19810, 281029, 290977, 291224, 291226, 672010, 824379]


In [42]:
importances = rf_clf.feature_importances_
feat_importance = pd.DataFrame({'feature': X.columns, 'importance': importances})
feat_importance = feat_importance.sort_values(by='importance', ascending=False)

print("Top features:\n", feat_importance.head())

Top features:
            feature  importance
10  FGRAPH_EIGEN11    0.161398
9   FGRAPH_EIGEN10    0.139988
8    FGRAPH_EIGEN9    0.124719
0    FGRAPH_EIGEN1    0.089089
6    FGRAPH_EIGEN7    0.087516
